In [1]:
from datetime import datetime, timedelta, timezone
from pybrew.analytics import base_alpha, base_beta, google_analytics_io, \
    google_analytics_view_id, to_dataframe, ga_target_audience_segment, business_cycle, parallel_test_groups, \
    git_sha_io, git_branch_io, unique_pageviews_of_sha_io, utc_time_from_sha_io, ga_segment_stats_io, ga_sha_segment, \
    unique_pageviews_of_segments_io, github_branch_sha_io, deep_get
from pybrew import pipe

print(datetime.utcnow().isoformat())

import json

def to_dict(x):
    return dict(zip(x.names, [list(y) for y in list(x) if y]))

def pprint(x):
    print(json.dumps(x, sort_keys=True, indent=4))
    
analytics = google_analytics_io()

2020-02-10T17:59:13.650033


In [2]:
%load_ext rpy2.ipython

# Split Test
### Are we sure that this branch will boost KPI?

We select "checkout visits" as a poxy KPI

- H0 - A and B have the same percent of `/checkout.html` visits
- Ha - B has more `/checkout.html` visits than A

We will use `power.prop.test` to estimate *sample* and *effect* sizes before starting the test

We will use `prop.test` to calculate p-value in the end of the test

## Alpha

In [3]:
alpha = base_alpha()
alpha

0.05

## Number of parallel testing groups

In [4]:
groups = parallel_test_groups()
groups

2

## Estimating baseline conversion and sample size

In [5]:
start = datetime.fromisoformat('2019-11-06')
end = start + business_cycle()

baseline = ga_segment_stats_io(
    analytics,
    start,
    end,
    [ga_target_audience_segment()]
)

print(f'''

Baseline is {baseline["n"]} visitors in a business cycle with conversion rate {baseline["conversion"]*100:.02f}%

''')



Baseline is 192 visitors in a business cycle with conversion rate 7.29%




## How many visitors should each group have to be significant?

In [6]:
min_n = baseline["n"] / groups
min_n

96.0

## What SHA and branches are we testing?

In [7]:
sha_a = github_branch_sha_io('master')
sha_b = git_sha_io()

print(sha_a, sha_b)

targets = {
    'A': {
        'sha': sha_a,
        'branch': 'master',
        'time': utc_time_from_sha_io(sha_a),
    },
    'B': {
        'sha': sha_b,
        'branch': git_branch_io(),
        'time': utc_time_from_sha_io(sha_b),
    }
}

targets

00d5e567c819f37632354d038563c51b0bb7fdea a8fcad65b32c2e82a60d31511572b3c84b1233a4


{'A': {'sha': '00d5e567c819f37632354d038563c51b0bb7fdea',
  'branch': 'master',
  'time': datetime.datetime(2020, 2, 10, 10, 53, 2, tzinfo=datetime.timezone.utc)},
 'B': {'sha': 'a8fcad65b32c2e82a60d31511572b3c84b1233a4',
  'branch': 'interactive_jupyter',
  'time': datetime.datetime(2020, 2, 10, 10, 46, 57, tzinfo=datetime.timezone.utc)}}

## Current results

In [8]:
analytics = google_analytics_io()

start = max(x['time'] for x in targets.values())
end = start + business_cycle()

def get_results_io(target, start, end):
    return ga_segment_stats_io(
        analytics,
        start,
        end,
        [ga_sha_segment(target['sha'])]
    )

results = {k: {**v, **{'results': get_results_io(v, start, end)}}  for k, v in targets.items()}
results

{'A': {'sha': '00d5e567c819f37632354d038563c51b0bb7fdea',
  'branch': 'master',
  'time': datetime.datetime(2020, 2, 10, 10, 53, 2, tzinfo=datetime.timezone.utc),
  'results': {'n': 3, 'n_conversion': 0, 'conversion': 0.0}},
 'B': {'sha': 'a8fcad65b32c2e82a60d31511572b3c84b1233a4',
  'branch': 'interactive_jupyter',
  'time': datetime.datetime(2020, 2, 10, 10, 46, 57, tzinfo=datetime.timezone.utc),
  'results': {'n': 0, 'n_conversion': 0, 'conversion': 0.0}}}

## Conclusion

In [9]:
def prop_test(a, b):
    a_successes = a['results']['n_conversion']
    a_trials = a['results']['n']
    
    b_successes = b['results']['n_conversion']
    b_trials = b['results']['n']
    
    assert a_trials > 0 and b_trials > 0, "No data, please wait till branches have visits"
    
    result = %R prop.test(c($a_successes, $b_successes), c($a_trials, $b_trials))
    return to_dict(result)

test_results = prop_test(results['A'], results['B'])

pprint(test_results)

assert test_results['p.value'][0] <= alpha
print("The test result are significant")

assert results["B"]["results"]["conversion"] > results["A"]["results"]["conversion"], \
    f'''Branch {results["B"]["branch"]} is worse than {results["A"]["branch"]}, we can reject the new brach'''

print(
f'''
Branch {results["B"]["branch"]} is better than {results["A"]["branch"]}! We can merge the new branch!
'''
)

AssertionError: No data, please wait till branches have visits